Para transformar las tuplas que tenemos o la info que tenemos en agrupados de los nodos con sus propiedades, sus embeddings y sus arcos.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict

In [ ]:
path_dir = ./'

In [ ]:
nodes = pd.read_pickle(path_dir + '__triples_nodes_pdd.pickle')
len(nodes)

In [ ]:
neo_nodes = defaultdict(dict)

for n in tqdm(nodes):
    neo_nodes[n[0]][n[1]] = n[2]

len(neo_nodes)

In [ ]:
del nodes

In [ ]:
for k,v in neo_nodes.items():
    print(k,v)
    print('-------------')
    break

In [ ]:
import pickle
with open(path_dir + 'neo4j_nodes.pickle','wb') as file:
    pickle.dump(neo_nodes,file)

In [ ]:
del neo_nodes

In [ ]:
drugs = pd.read_pickle(path_dir + '__triples_drugs_pdd.pickle')
len(drugs)

In [ ]:
drug_embeddings = pd.read_pickle(path_dir + 'drugs_emb.pk')
len(drug_embeddings)

In [ ]:
neo_nodes_drugs = defaultdict(dict) # TODO: This was relations two!!

drug_interactions = defaultdict(set)

for d in tqdm(drugs):
    
    if d[1] == 'interacts with':
        drug_interactions[d[0]].add(d[2])
        continue
    
    neo_nodes_drugs[d[0]][d[1]] = d[2]
    
for k in neo_nodes_drugs:
    if k not in drug_embeddings:
        continue
    
    for t,e in drug_embeddings[k].items():
        neo_nodes_drugs[k][t+'__embeddings'] = e
    
    
len(neo_nodes_drugs)

In [ ]:
len(drug_interactions)

In [ ]:
with open(path_dir + 'neo4j_nodes_drugs.pickle','wb') as file:
    pickle.dump(neo_nodes_drugs,file)

In [ ]:
icd = pd.read_pickle(path_dir + '__triples_icds_pdd.pickle')
len(icd)

In [ ]:
icd_embeddings = pd.read_pickle(path_dir + 'icds_emb.pk')
len(icd_embeddings)

In [ ]:
neo_nodes_icds = defaultdict(dict)

for i in icd:
    
    if i[1] == 'is child of': # rel
        continue
        
    if i[1] == 'expanded form of': # nothing
        continue 
            
    neo_nodes_icds['ICD9CM/'+i[0]][i[1]] = i[2] # has type, is known as -- el ICD es para que haga matching con el otro
    if k in icd_embeddings:
        for t,e in icd_embeddings[i[0]].items():
            neo_nodes_icds['ICD9CM/'+i[0]][t+'__embeddings'] = e
    
len(neo_nodes_icds)

In [ ]:
with open(path_dir + 'neo4j_nodes_icds.pickle','wb') as file:
    pickle.dump(neo_nodes_icds,file)

In [ ]:
edges = pd.read_pickle(path_dir + '__triples_edges_pdd.pickle')
len(edges)

In [ ]:
edges_drug_interactions = defaultdict(set)

for e in tqdm(edges):
    if e[1] == 'has prescription':
        continue
    
    if e[1] == 'was admitted in':
        continue
    
    if e[1] == 'takes drug':
        continue
    
    if e[1] == 'was diagnosed':
#         print(e)
        continue
    
    if e[1] == 'interacts with':
        edges_drug_interactions[e[0]].add(e[2])
        continue
    
    print(e)
    break

In [ ]:
for k, i in tqdm(drug_interactions.items()):
    if k in edges_drug_interactions:
        for ii in i:
            if ii not in edges_drug_interactions[k]:
                edges.append((k,'interacts with',ii))
    else:
        for ii in i:
            edges.append((k,'interacts with',ii))

In [ ]:
icd9 = pd.read_pickle(path_dir + 'icd9_umls.pickle')
icd9

for k,v in tqdm(icd9.items()):
    if v['type_'] != 'ICD9CM':
        continue
    
    if 'subclass' in v:
        edges.append(('ICD9CM/'+k,'is subclass of','ICD9CM/'+v['subclass']))

In [ ]:
# reconversión de los edges a dictionary para levantarlos directo en neo4j
from collections import deque
edges_dict = deque()
for e in tqdm(edges):
    edges_dict.append({'source':e[0],'dest':e[2],'rel_type':e[1]})

len(edges_dict)

In [ ]:
with open(path_dir + 'neo4j_edges.pickle','wb') as file:
    pickle.dump(edges_dict,file)